### Definisi Kelas
> 
* Saat lisensi tidak ditemukan
* Saat projeksi referensi error

In [1]:
class LicenseError(Exception):
    pass

class SpatialRefProjError (Exception):
    pass

### Import Library yang diperlukan
> 
* arcpy sebagai tools geoprocessing utama
* pandas dan numpy sebagai tools manipulasi daframe dan matriks

In [2]:
import arcpy as ap
import pandas as pd
import numpy as np
import os, sys, time, glob, math, string

### Fungsi untuk checkin dan checkout
> 
* setiap fungsi pada arcgis memerlukan lisensi saat dijalankan di area luar desktop
* checkout digunakan saat mulai memanggil ekstensi yang diinginkan dan checkin saat fungsi telah selesai

In [3]:
##  Checkout ArcGIS extension by name
def checkout_Ext(ext_type):
    if ap.CheckExtension(ext_type) == 'Available':
        ap.CheckOutExtension(ext_type)
        print "\nChecking out " + ext_type + " Extension"
    else:
        raise LicenseError
        print "\nCannot checkout " + ext_type + " Extension"


##  Checkin ArcGIS extension by name
def checkin_Ext(ext_type):
    ap.CheckInExtension(ext_type)
    print "\nChecking in " + ext_type + " Extension"

### Fungsi mengambil nomer band
> 
* setiap nama file data akan diambil nomer bandnya saja

In [4]:
def band_nmbr(filename):

    try:
        band_num = int(filename.split('_')[1].split('.')[0].replace('B', ''))
        return band_num
    except:
        pass

### Fungsi membaca metadata
> 
* data dapat dipanggil dengan double bracket dengan urutan [group][atribut]

In [5]:
def readMetadata(metadataFile):

    f = metadataFile
    
    #Create an empty dictionary with which to populate all the metadata fields.
    metadata = {}

    #Each item in the txt document is seperated by a space and each key is
    #equated with '='. This loop strips and seperates then fills the dictonary.

    for line in f:
        if not line.strip() == "END":
            val = line.strip().split('=')
            metadata [val[0].strip()] = val[1].strip().strip('"')      
        else:
            break

    return metadata

### Fungsi reprojeksi
> 
* disesuaikan dengan projeksi yang diinginkan
* langkah awal untuk melakukan perhitungan TOA reflectance

In [6]:
def reproject(input_dir, output_dir, projection, meta):
    #  Setting output directory id not defined
    ap.env.workspace = input_dir
    if os.path.exists(output_dir):
        os.system('rmdir /s /q '+ output_dir)
        #sys.exit(0)
        print "\nDirectory for reprojection already Exisits"
        os.mkdir(output_dir)
    else:
        os.mkdir(output_dir)
        print "\nCreated the output directory: " + output_dir  

    rasters = ap.ListRasters('*.TIF')
    ms_bands = [band for band in rasters if (band_nmbr(band) >= 4 and band_nmbr(band) <=6)]
    bqa_band = [band for band in rasters if (band_nmbr(band) == None)][0]
    #bqa_band = ms_bands[0]
    try:
        #checkout_Ext("Spatial")
        print "\nReprojecting and Cleaning landsat bands."
        ap.AddMessage("\nReprojecting and Cleaning landsat bands.")
        for band in ms_bands:
            print "\nReclassifying NoData for band " + band
            ap.AddMessage("\nReclassifying NoData for band " + band)
            outCon = ap.sa.Con(ap.sa.Raster(bqa_band) != 1, ap.sa.Raster(band))
            out_band = os.path.join(output_dir, band)

            print "Reprojecting band"
            ap.AddMessage("Reprojecting band")
            ap.ProjectRaster_management(outCon, out_band, projection)

    except SpatialRefProjError:
        ap.AddError ("Spatial Data must use a projected coordinate system to run")

    except LicenseError:
        ap.AddError ("Spatial Analyst license is unavailable")
        
    finally:
        checkin_Ext("Spatial")

### Fungsi konversi digital number to TOA reflectance

In [7]:
def calc_toa(input_dir, output_dir, meta):
    '''Raster Algebra to run reflectance equation

        @param   input_dir: landsat 8 directory after unzip
        @type    input_dir: c{str}
        @param   meta: metadata parsed from MTL file
        @type    meta: dictionary
        @return  output: out raster path
    '''

    ##  Setting output directory id not defined
    ap.env.workspace = input_dir

    if output_dir is None:
        output_dir = os.path.join(input_dir, "TOA")
        if os.path.exists(output_dir):
            sys.exit(0)
            print "\nDirectory for reprojection already Exisits"
            ap.AddMessage("\nDirectory for reprojection already Exisits")
        else:
            os.mkdir(output_dir)
            print "\nCreated the output directory: " + output_dir
            ap.AddMessage("\nCreated the output directory: " + output_dir)
    else:
        if os.path.exists(output_dir):
            sys.exit(0)
            print "\nDirectory for reprojection already Exisits"
            ap.AddMessage("\nDirectory for reprojection already Exisits")
        else:
            os.mkdir(output_dir)
            print "\nCreated the output directory: " + output_dir
            ap.AddMessage("\nCreated the output directory: " + output_dir)

    rasters = ap.ListRasters('*.TIF')
    ms_bands = [band for band in rasters if (band_nmbr(band) != None)]

    try:

        #checkout_Ext("Spatial")

        print "\nCalculating TOA Reflectance for landsat 8 bands"
        ap.AddMessage("\nCalculating TOA Reflectance for landsat 8 bands")
        for band in ms_bands:

            print "\nBegining to calculate TOA for " + band
            ap.AddMessage("\nBegining to calculate TOA for " + band)
            number = band_nmbr(band)
            raster_band = ap.sa.Raster(band)
            out_band = str(meta['L1_METADATA_FILE']['LANDSAT_SCENE_ID']) + 'TOA_B' + str(number) + '.img'
        
            sun_elev = float(meta['IMAGE_ATTRIBUTES']['SUN_ELEVATION'])
            rad_mult = float(meta['RADIOMETRIC_RESCALING']['RADIANCE_MULT_BAND_' + str(number)])
            rad_add = float(meta['RADIOMETRIC_RESCALING']['RADIANCE_ADD_BAND_' + str(number)])
        
            toa_refl = (rad_mult*raster_band + rad_add)/(math.sin(sun_elev))

            print "Writing " + str(out_band)
            ap.AddMessage("Writing " + str(out_band))
            toa_refl.save(os.path.join(output_dir, out_band))

    except SpatialRefProjError:
        ap.AddError ("Spatial Data must use a projected coordinate system to run")

    except LicenseError:
        ap.AddError ("Spatial Analyst license is unavailable") 	

    finally:
        checkin_Ext("Spatial")

In [8]:
dataPath = "D:\FloodDetect\Clone1"
projectedPath = "D:\FloodDetect\Clone1\processed"
toaPath = "D:\FloodDetect\Clone1\RefToa"
metadataPath = "D:\FloodDetect\Clone1\LC81210602015188RPI00_MTL.txt"
projection = "PROJCS['WGS_1984_ARC_System_Zone_13',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Equidistant_Cylindrical'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',-60.32378942],UNIT['Meter',1.0]];-9920700 -10018900 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision"

In [9]:
metadataFile = open(metadataPath)
metadata = readMetadata(metadataFile)

In [10]:
reproject(dataPath, projectedPath, projection , metadata)


Created the output directory: D:\FloodDetect\Clone1\processed

Reprojecting and Cleaning landsat bands.

Reclassifying NoData for band LC81210602015188RPI00_B4.TIF
Reprojecting band

Reclassifying NoData for band LC81210602015188RPI00_B5.TIF
Reprojecting band

Reclassifying NoData for band LC81210602015188RPI00_B6.TIF
Reprojecting band

Checking in Spatial Extension


In [11]:
#calc_toa(projectedPath, toaPath, metadata)